# Урок 3. Системы управления базами данных MongoDB и SQLite в Python

1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД (без датафрейма)  
2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)  
3) Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта  

In [8]:
import requests
from pprint import pprint
import re  # потребуется для разбора строки регулярным выражением
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
from pymongo import errors

In [9]:
client = MongoClient('localhost', 27017)
db = client['vacancies_db']

In [10]:
def put_vacancy_to_db(vacancy_data):
    try:
        db.vacancies.insert_one(vacancy_data)
    except errors.DuplicateKeyError:
        pass
        return 0
    else:
        return 1

In [11]:
def get_vacancy_compensation(vacancy_comp):
    
    vacancy_comp = re.sub('(?<=\d)\s(?=\d)', '', vacancy_comp)  # убрать пробелы между числами
    vacancy_comp = vacancy_comp.lower()
    words = re.findall(r'\w+', vacancy_comp)  # составит список слов из строки

    min_compensation, max_compensation, currency_compensation = None, None, None

    if words:
        if len(words) > 2:
            currency_compensation = words[2]

        if words.count('от') != 0:
            min_compensation = words[1]
        elif words.count('до') != 0:
            max_compensation = words[1]
        elif words.count('по') != 0:
            pass
        else:
            min_compensation = words[0]
            max_compensation = words[1]

    return min_compensation, max_compensation, currency_compensation

In [12]:
main_link_hh = 'https://hh.ru/'
job_title = 'Data+scientist'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36'}

response = requests.get(main_link_hh + f'/search/vacancy?text={job_title}', headers=headers)

soup = bs(response.text,'lxml')

new_vacancy = 0

In [13]:
vacancy_block = soup.find('div',{'class':'vacancy-serp'})
vacancy_list = vacancy_block.find_all('div', {'class':'vacancy-serp__vacancy'})
vacancy_list = vacancy_block.findChildren('div', {'class':'vacancy-serp-item'}, recursive=False)

In [14]:
vacancies = []

for vacancy in vacancy_list:
    vacancy_data = {}
    
    site_name = main_link_hh
    vacancy_name = vacancy.find('a').getText()
    vacancy_link = vacancy.find('a')['href']
    employer = vacancy.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).getText()
    location = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-address'}).getText()
   
    vacancy_data['name'] = vacancy_name
    vacancy_data['link'] = vacancy_link
    vacancy_data['employer'] = employer
    vacancy_data['location'] = location
    vacancy_data['site'] = site_name
    
    vacancy_comp = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText().replace(u'\xa0', u'')
    vacancy_data['min_compensation'], vacancy_data['max_compensation'], vacancy_data['currency_compensation'] = get_vacancy_compensation(vacancy_comp)

    new_vacancy += put_vacancy_to_db(vacancy_data)
    
    vacancies.append(vacancy_data)

In [ ]:
compensation = 100000

vacancies = db.hh.find({'$or' : [{'min_compensation': {'$gt' : compensation}}, {'max_compensation': {'$gt' : compensation}}]})\
                 .limit(5)

for vacancy in vacancies:
    print(vacancy)